# Web Feature Services

Web Feature Services (WFS) provide access to geospatial data in a more interactive and granular manner compared to Web Map Services (WMS). Instead of pre-rendered image tiles, WFS allows users to retrieve and manipulate individual features or objects on a map.

## Feature Retrieval

With WFS, users can query and retrieve specific features based on their spatial and attribute properties. The service provides a standardized interface for accessing and interacting with geospatial data, making it easier to integrate and analyze the information in various applications.

## Feature Specifications

Just like WMS, WFS also follows specific specifications that define its functionality and usage. The main specification for WFS is the OGC WFS standard (Open Geospatial Consortium Web Feature Service), which outlines the operations and formats supported by the service.

The URL pattern for accessing WFS data typically includes parameters for specifying the requested feature type, output format, and any additional query parameters. Here's an example URL pattern:

```
http://example.com/wfs?service=WFS&version=1.1.0&request=GetFeature&typeName=layer_name&outputFormat=geojson
```

In this example, the URL includes parameters such as `service` (specifying the WFS service), `version` (specifying the WFS version), `request` (specifying the type of request, such as GetFeature), `typeName` (specifying the name of the layer or feature type to retrieve), and `outputFormat` (specifying the desired format of the returned data, such as GeoJSON).

## Online WFS Viewers

Online WFS viewers provide a user-friendly interface for accessing and interacting with geospatial data through web feature services.

Here are some examples of online WFS viewers:

NOAA Weather Viewer (https://viewer.weather.noaa.gov/): This viewer, provided by the National Oceanic and Atmospheric Administration (NOAA), allows users to access and visualize real-time weather information through web feature services. Users can query and retrieve specific weather features, such as radar data, forecasts, and satellite imagery, and overlay them with their own data for comprehensive weather analysis.

National Map Viewer (https://apps.nationalmap.gov/viewer/): The National Map Viewer, developed by the United States Geological Survey (USGS), offers access to a wide range of geospatial data through web feature services. Users can retrieve and overlay various datasets, including topographic maps, aerial imagery, and elevation data, to explore and analyze different geographic features in combination with their own data.

While existing online WFS viewers provide access to diverse geospatial data, the true power of web feature services (WFS) lies in the ability to overlay them with your own data. This capability empowers users to integrate and analyze geospatial information from multiple sources, facilitating advanced spatial analysis, decision-making, and data-driven insights.

We will explore how to utilize this functionality further in the following sections.

## Prerequisites

The following packages are good to know, but not required, as we will be walking through them below.


| Concepts | Importance | Notes |
| --| --| --- |
| [Intro to hvPlot](https://hvplot.holoviz.org/) | Helpful | High-level plotting |
| [Intro to GeoViews](https://geoviews.org/) | Helpful | Geographic visualizations |
| [Intro to Geopandas](https://geopandas.org/) | Helpful | GIS files |
| [Intro to Panel](https://panel.holoviz.org/) | Helpful | Dashboard creations |
| [Intro to Requests](https://requests.readthedocs.io/en/latest/) | Helpful | Webpage requests |
| [Intro to Cartopy](https://foundations.projectpythia.org/core/cartopy/cartopy.html) | Helpful | Map projections |
| [Intro to OWSLib](https://owslib.readthedocs.io/en/latest/usage.html) | Helpful | WFS URLs |

- **Time to learn**: 10 minutes

---

## Imports

Let's start by importing a few packages:

GeoViews is a Python library that simplifies working with geographic data and enables the integration of geographical features to your maps.

Hvplot is a plotting library built on top of GeoViews. It provides a concise and intuitive API for generating interactive plots and charts from geospatial data.

CartoPy is a Python library specifically designed for map projections and geospatial data visualization. It offers a wide range of tools and functionalities to work with geospatial datasets, making it easier to generate maps with different projections.

Panel is a powerful Python library that allows you to create interactive dashboards and applications. It provides flexible tools for building custom user interfaces with interactive controls, widgets, and layout components, enhancing the interactivity of your visualizations and data analysis workflows.

Requests is a popular Python library for making HTTP requests. It simplifies sending HTTP requests to web servers and handling the responses. In the context of geospatial data visualization, requests can be used to retrieve data from web services, such as WMS (Web Map Service) endpoints, to integrate external data sources into your visualizations.

Geopandas is a Python library used for working with geospatial data. With Geopandas, you can easily load shapefiles into a GeoDataFrame, which is a tabular data structure that combines the spatial information of the shapefile with its attribute data. This allows you to perform various geospatial operations and analyses on the data.

OWSLib is a Python library designed for client-side programming using the interface standards of the Open Geospatial Consortium (OGC) web services and their associated content models. Specifically, in this scenario, OWSLib will be utilized solely for the purpose of constructing URLs for WFS.

To enable the Bokeh plotting backend for GeoViews, we use the line `gv.extension("bokeh")`. GeoViews supports multiple plotting backends, including Bokeh and Matplotlib, giving you the flexibility to choose the one that suits your requirements best.

Lastly, `pn.extension()` initializes the Panel library and sets up the necessary environment for creating interactive panels and dashboards. You can also specify configurations such as `sizing_mode="stretch_width"` within `pn.extension()`.

In [ ]:
import geoviews as gv
import hvplot.pandas
import cartopy.crs as ccrs
import panel as pn
import requests
import shapely
import geopandas as gpd
from owslib.wfs import WebFeatureService

gv.extension("bokeh")
pn.extension()

## Accessing WFS Features

Unlike WMS, where built-in sources exist, accessing WFS requires utilizing external sources. However, similar to WMS, there are several external sources available for WFS, such as NOAA, which can be easily accessed. Depending on the specific source, accessing WFS data may also be free of charge.

For instance, if you prefer to obtain the CPC 6-10 day outlook as a GeoJSON instead of a tile image, you can utilize the gpd.read_file function, providing the relevant URL, or building it with `OWSLib`.

In [ ]:
def get_wfs_feature(base_resource_url):
    wfs = WebFeatureService(base_resource_url, version="2.0.0")
    wfs_contents = wfs.contents
    wfs_layer = list(wfs_contents)[0]
    feature = wfs.getfeature(
        typename=[wfs_layer],
        outputFormat="GeoJson",
    )
    return feature

base_resource_url = "https://mapservices.weather.noaa.gov/vector/services/outlooks/cpc_6_10_day_outlk/MapServer/WFSServer?service=WFS&version=1.0.0&request=GetFeature&srsname=EPSG%3A4326&typename=cpc_6_10_day_outlk%3ACPC_6-10_Day_Temperature_Outlook&propertyname=%2A"
feature = get_wfs_feature(base_resource_url)
gdf = gpd.read_file(feature)
gdf["geometry"] = gdf["geometry"].map(lambda polygon: shapely.ops.transform(lambda x, y: (y, x), polygon))  # flip x and y
gdf.head()

## Visualizing WFS Data

Now that it's serialized into a `gpd.GeoDataFrame`, visualizing the data is a trivial task with `hvplot`! Be sure to set `geo=True` to specify this is a geographic dataset.

In [ ]:
temp_outlook = gdf.hvplot(geo=True).opts(global_extent=True) * gv.feature.coastline()
temp_outlook

You can also add tooltips and colors to show the category and probability upon hover!

In [ ]:
temp_outlook = (
    gdf.hvplot(
        hover_cols=["cat", "prob"],
        tools=["hover"],
        cmap=["#ff6666", "#66b3ff", "#999999"],
        geo=True,
    ).opts(global_extent=True)
    * gv.feature.coastline()
)
temp_outlook

## Overlaying Custom Data

By overlaying WFS layers with your own data, you can gain insights into the outlook for specific locations.

For instance, on its own, the coordinates (40.1N, -88.26W) may not mean much, but by overlaying WFS features, you now know the temperature outlook for that location.

In [ ]:
coord_point = gv.Points((-88.24, 40.1), crs=ccrs.PlateCarree()).opts(
    color="yellow", size=10, marker="x"
)

temp_outlook * coord_point

## Summary

Web Feature Services (WFS) offer an interactive and granular way to access geospatial data.

With WFS, users can retrieve specific features based on their properties.

It follows the OGC WFS standard and allows for the integration of various datasets.

Online WFS viewers provide user-friendly interfaces for accessing and visualizing geospatial data.

By overlaying WFS layers with custom data, users can gain valuable insights and unlock the full potential of geospatial information.
